# PythonでBM25を計算する方法（tf-idfの場合との比較）

本コンテンツは、以下の記事に付属するJupyterノートブックです。もちろんColabなどでも実行可能です。[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/isshiki/MLnotebooks/blob/master/How_to_calc_BM25_by_python.ipynb)

- [記事「BM25／Okapi BM25（情報検索のアルゴリズム）とは？：AI・機械学習の用語辞典 - ＠IT」](https://atmarkit.itmedia.co.jp/ait/articles/2404/22/news021.html)に付属するノートブックです。

下記の記事の説明をベースに実装しています。実装はChatGPTを使って楽しました。

- [記事「tf-idf（term frequency - inverse document frequency）とは？：AI・機械学習の用語辞典 - ＠IT」](https://atmarkit.itmedia.co.jp/ait/articles/2112/23/news028.html)

## 表1　各文書における各単語の出現回数（BoW：Bag of Words）

In [11]:
import pandas as pd
from collections import Counter

# 各文書のデータ
documents = {
    '文書A': 'イヌ イヌ イヌ サル キジ',
    '文書B': 'イヌ ネコ ネコ キツネ',
    '文書C': 'イヌ タヌキ キツネ'
}

# 単語の出現回数をカウントしDataFrameに変換する
data = {}
for doc, text in documents.items():
    word_count = Counter(text.split())
    data[doc] = word_count

# DataFrameを作成し、NaNを0に変換（単語が出現しなかった場合は0とする）
df = pd.DataFrame(data).fillna(0).astype(int).T

# 列の順序を指定
columns_order = ['イヌ', 'キジ', 'キツネ', 'サル', 'タヌキ', 'ネコ']
df = df[columns_order]

# 列と行の名前を設定
df.columns.name = '単語'
df.index.name = '文書'

# 表示
df

単語,イヌ,キジ,キツネ,サル,タヌキ,ネコ
文書,,,,,,
文書A,3,1,0,1,0,0
文書B,1,0,1,0,0,2
文書C,1,0,1,0,1,0


## 表2　各文書における各単語の出現頻度（tf：term frequency）

In [12]:
# tf値を計算（各要素を行の合計で割る）
tf = df.div(df.sum(axis=1), axis=0)

# 列と行の名前を設定
tf.columns.name = '単語'
tf.index.name = '文書'

# 表示
print('tf値（tf-idf）')
tf.round(2)

tf値（tf-idf）


単語,イヌ,キジ,キツネ,サル,タヌキ,ネコ
文書,,,,,,
文書A,0.60,0.2,0.00,0.2,0.00,0.0
文書B,0.25,0.0,0.25,0.0,0.00,0.5
文書C,0.33,0.0,0.33,0.0,0.33,0.0


### Okapi BM25の正規化されたtf値の場合

In [13]:
# パラメータ
k1 = 2.0
b = 0.75

# 各文書の長さ（単語数）と平均文書長
document_lengths = df.sum(axis=1)
average_length = document_lengths.mean()

# BM25のtf値を各単語に対して計算
k_values = k1 * (1 - b + b * document_lengths / average_length)
bm25_tf = (df * (k1 + 1)) / (df + k_values.values[:, None])  # 各単語のTF値を計算

# 表示
print('tf値（BM25）')
bm25_tf.round(2)

tf値（BM25）


単語,イヌ,キジ,キツネ,サル,タヌキ,ネコ
文書,,,,,,
文書A,1.67,0.89,0.00,0.89,0.00,0.0
文書B,1.00,0.00,1.00,0.00,0.00,1.5
文書C,1.14,0.00,1.14,0.00,1.14,0.0


## 表3　各単語を含む文書の数

In [14]:
# 各単語が含まれる文書の数を計算
document_count = (df > 0).sum(axis=0)

# 単一行のDataFrameを作成
document_count = pd.DataFrame([document_count], index=["全文書（3個）中での文書の数"])

# 表示
document_count

単語,イヌ,キジ,キツネ,サル,タヌキ,ネコ
全文書（3個）中での文書の数,3,1,2,1,1,1


## 表4　各単語の文書間でのレア度（idf：inverse document frequency）

In [15]:
import numpy as np

# 文書の総数
N = len(df)

# 各単語が含まれる文書の数を計算
document_count = (df > 0).sum(axis=0)

# idf値を計算
idf = np.log(N / document_count)

# 単一行のDataFrameとしてidf値を表示
idf = pd.DataFrame([idf], index=["idf値（tf-idf）"])

# 表示
idf.round(2)

単語,イヌ,キジ,キツネ,サル,タヌキ,ネコ
idf値（tf-idf）,0.0,1.1,0.41,1.1,1.1,1.1


### Okapi BM25のidf値の場合

In [16]:
import numpy as np
import pandas as pd

# 文書の総数
N = len(df)

# 各単語が含まれる文書の数を計算
document_count = (df > 0).sum(axis=0)

# BM25のidf値を計算
idf_bm25 = np.log(((N - document_count + 0.5) / (document_count + 0.5)) + 1)

# 負の値を避けるために、この例では「最小値を0に制限」
idf_bm25 = np.maximum(idf_bm25, 0)

# 単一行のDataFrameとしてidf値を表示、少数第2位まで丸める
bm25_idf = pd.DataFrame([idf_bm25], index=["idf値（BM25）"]).round(2)

# 表示
bm25_idf

単語,イヌ,キジ,キツネ,サル,タヌキ,ネコ
idf値（BM25）,0.13,0.98,0.47,0.98,0.98,0.98


## 表5　各文書における各単語の重要度（tf-idf）

In [17]:
# idf値のDataFrameから必要なデータを取得
idf_series = idf.loc["idf値（tf-idf）"]

# tf値とidf値を掛け合わせてtf-idf値を計算
tf_idf = tf.mul(idf_series, axis=1)

# 表示
print('tf-idf値')
tf_idf.round(2)

tf-idf値


単語,イヌ,キジ,キツネ,サル,タヌキ,ネコ
文書,,,,,,
文書A,0.0,0.22,0.00,0.22,0.00,0.00
文書B,0.0,0.00,0.10,0.00,0.00,0.55
文書C,0.0,0.00,0.14,0.00,0.37,0.00


### Okapi BM25の場合

In [18]:
# idf値のDataFrameから必要なデータを取得
bm25_idf_series = bm25_idf.loc["idf値（BM25）"]

# 正規化されたtf値とidf値を掛け合わせてBM25スコアを計算
bm25_tf_idf = bm25_tf.mul(bm25_idf_series, axis=1)

# 表示
print('BM25スコア')
bm25_tf_idf.round(2)

BM25スコア


単語,イヌ,キジ,キツネ,サル,タヌキ,ネコ
文書,,,,,,
文書A,0.22,0.87,0.00,0.87,0.00,0.00
文書B,0.13,0.00,0.47,0.00,0.00,1.47
文書C,0.15,0.00,0.54,0.00,1.12,0.00


## 表6　各文書の特徴ベクトル同士で計算したコサイン類似度の一覧表


In [19]:
from sklearn.metrics.pairwise import cosine_similarity

# tf-idf値のDataFrameを用いてコサイン類似度を計算
cosine_sim_matrix = cosine_similarity(tf_idf)

# コサイン類似度の結果をDataFrameに変換
cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=tf_idf.index, columns=tf_idf.index)

# 表示
print('類似度（tf-idf）')
cosine_sim_df.round(2)

類似度（tf-idf）


文書,文書A,文書B,文書C
文書,,,
文書A,1.0,0.00,0.00
文書B,0.0,1.00,0.06
文書C,0.0,0.06,1.00


### Okapi BM25の場合

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

# BM25スコアのDataFrameを用いてコサイン類似度を計算
bm25_cosine_sim_matrix = cosine_similarity(bm25_tf_idf)

# コサイン類似度の結果をDataFrameに変換
bm25_cosine_sim_df = pd.DataFrame(bm25_cosine_sim_matrix, index=bm25_tf_idf.index, columns=bm25_tf_idf.index)

# 表示
print('類似度（BM25）')
bm25_cosine_sim_df.round(2)

類似度（BM25）


文書,文書A,文書B,文書C
文書,,,
文書A,1.00,0.01,0.02
文書B,0.01,1.00,0.14
文書C,0.02,0.14,1.00
